# Ejemplos de Aplicacion de la Optimizacion sin restricciones

In [5]:
import numpy as np

## 1. Diseño de filtro de respuesta finita (FIR)
https://link.springer.com/chapter/10.1007/978-1-4612-0571-5_5

Denotamos por $h_0,\ h_1,\ldots,\ h_n$ los coeficientes de un filtro de respuesta finita (FIR) de longitud $n+1$. Esto significa que la salida del filtro esta dada por
$$ y[k] = \sum_{k=0}^n h_iu[k-i].
$$
La respuesta en frecuencia del filtro es la función
$$ H[\omega] = \sum_{k=0}^n h_k e^{-jk\omega},\text{ con } \omega\in [0,\pi],
$$
donde $j=\sqrt{-1}$ y $\omega$ la variable de velocidad angular. La expresión anterior la podemos reescribir como el siguiente producto punto
$$ H[\omega_1] = [1,e^{-j(1)\omega_1},\ldots,e^{-j(n)\omega_1}] \cdot [h_0,h_1,\ldots,h_n],
$$
y de forma mas general, como el siguiente producto matricial
$$ \mathbf{H} = \left[\begin{array}{ccccc}
	1 & \mathrm{e}^{-j \omega_1} & \mathrm{e}^{-j 2 \omega_1} & \cdots & \mathrm{e}^{-j(n) \omega_1} \\
	1 & \mathrm{e}^{-j \omega_2} & \mathrm{e}^{-j 2 \omega_2} & \cdots & \mathrm{e}^{-j(n) \omega_2} \\
	\vdots & \vdots & \vdots & & \vdots \\
	1 & \mathrm{e}^{-j \omega_N} & \mathrm{e}^{-j 2 \omega_N} & \cdots & \mathrm{e}^{-j(n) \omega_N}
\end{array}\right] \left[\begin{array}{c}
h_0 \\
h_1 \\
\vdots \\
h_{n}
\end{array}\right] = \mathbf{A} \mathbf{h}.
$$
donde $\mathbf{H}$ es un vector con $N$ elementos, $\mathbf{A} \in \mathbb{R}^{N\times (n+1)}$ y $\mathbf{h}\in \mathbb{R}^{n+1}$ es un vector compuesto por los coeficiente $h_k$. Buscamos minimizar la diferencia entre la respuesta en frecuencia deseada ($\mathbf{H}^{\text{des}}$)  con la generada a partir del filtro que estamos sintonizando

$$  \underset{\mathbf{h}}{\text{minimizar }}
\left\|\left[\begin{array}{ccccc}
	1 & \mathrm{e}^{-j \omega_1} & \mathrm{e}^{-j 2 \omega_1} & \cdots & \mathrm{e}^{-j(n) \omega_1} \\
	1 & \mathrm{e}^{-j \omega_2} & \mathrm{e}^{-j 2 \omega_2} & \cdots & \mathrm{e}^{-j(n) \omega_2} \\
	\vdots & \vdots & \vdots & & \vdots \\
	1 & \mathrm{e}^{-j \omega_N} & \mathrm{e}^{-j 2 \omega_N} & \cdots & \mathrm{e}^{-j(n) \omega_N}
\end{array}\right]\left[\begin{array}{c}
	h_0 \\
	h_1 \\
	\vdots \\
	h_{n}
\end{array}\right]-\left[\begin{array}{c}
	H_1^{\text {des }} \\
	H_2^{\text {des }} \\
	\vdots \\
	H_N^{\text {des }}
\end{array}\right]\right\|^2,
$$
o de manera matricial

$$ \underset{\mathbf{h}}{\text{minimizar}}
\left\|\mathbf{A} \mathbf{h}- \mathbf{H}^{\text{des}}\right\|^2.
$$
Debido que la función objetivo pertenece a los números complejos, se puede reescribir la ecuación anterior como

$$ \underset{\mathbf{h}}{\text{minimizar}}
\left\| \text{real}(\mathbf{A} \mathbf{h}- \mathbf{H}^{\text{des}})\right\|^2+\left\| \text{imag}(\mathbf{A} \mathbf{h}- \mathbf{H}^{\text{des}})\right\|^2.
$$


In [ ]:
import numpy as np

n = 20 #Cantidad de coeficientes

N = 15*n #Numero de frecuencias

#Vector columna de frecuencias
w = np.linspace(0,np.pi,N).reshape(-1, 1)

# Respuesta en frecuencia del filtro deseado
D = 5.25
Hdes = np.exp(-1j*D*w)

# Construccion de la matriz A
A = np.exp( -1j * np.kron(w, np.arange(n)))

#Extraer parte real e imaginaria
Hdes_R = np.real(Hdes)
Hdes_I = np.imag(Hdes)

A_R = np.real(A)
A_I = np.imag(A)

def ObjFun(h):
  return np.sum( np.square(np.matmul(A_R,h) - Hdes_R) + np.square(np.matmul(A_I,h) - Hdes_I))

def gradObjFun(h):
  return 2*A_R.T@(np.matmul(A_R,h) - Hdes_R) + 2*A_I.T@(np.matmul(A_I,h) - Hdes_I)

## Actividades

* A partir de la función objetivo y su gradiente, usar el método del gradiente más descendiente (Steepest Descent), para encontrar la solución al problema de optimización.
* Seleccionar un valor adecuado del paso ($\alpha$), de tal manera que el proceso de optimización converja a buenas soluciones.
* Comparar con el método de Newton.

In [1]:
#Hacer codigo aqui

## 2. Estimación de parámetros Aerogenerador
![](https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41598-023-36458-w/MediaObjects/41598_2023_36458_Fig1_HTML.png)
https://www.nature.com/articles/s41598-023-36458-w

In [39]:
import jax.numpy as jnp
from jax import grad, random

Los datos incluidos en el articulo son:

In [32]:
v = jnp.array([0.126, 0.799, 1.287, 1.749, 2.259, 2.719, 3.284, 3.751, 4.246,
              4.745, 5.244, 5.724,  6.234, 6.738, 7.243, 7.728, 8.232, 8.741,
              9.219, 9.704, 10.230, 10.724, 11.211, 11.767, 12.245, 12.715,
              13.226, 13.743, 14.300, 14.696, 15.232, 15.649, 16.029, 16.803,
              17.049, 17.885])
P = jnp.array([-3.151, -3.181, -3.213, -3.187, 2.824, 20.331, 74.153, 132.315, 225.115,
              347.751, 486.502, 660.094, 838.259, 1039.989, 1229.457, 1411.141, 1540.810, 1681.272,
              1783.571, 1818.255, 1822.494, 1821.436, 1838.905, 1840.394, 1838.213, 1839.370,
              1838.915, 1839.852, 1840.354, 1840.536, 1840.206, 1840.670, 1839.220, 1838.905,
              1829.330, 1837.585])

De acuerdo con la cita de arriba, la curva de funcionamiento de un aerogenerador se puede aproximar con la siguiente función de 4 parámetros:
$$ P(v, \boldsymbol{\theta}) = a\left(\frac{1+b\exp(-v/d)}{1+c\exp(-v/d)}\right),
$$
donde $a$ es el valor máximo, y los otros parámetros sirven para ajustar la forma de la curva. El problema de optimización es encontrar los valores de los 4 parámetros $\boldsymbol{\theta} = [a,b,c,d]$, que mejor describan los datos medidos de la potencia y velocidad del aero-genreador.

In [34]:
def Power(params, v): #Potencion como funcion de la velocidad del viento
  a,b,c,d = params
  temp = jnp.exp(-v/d)
  return a*(1.+b*temp)/(1.+c*temp)

La función objetivo se describe a partir del error cuadrático entre la potencia medida y la potencia aproximada, así

$$ \underset{\boldsymbol{\theta}}{ \text{min } } \sum_{i=1}^{n} \left(P_i - P(v_i,\boldsymbol{\theta})\right)^2,
$$
donde $P_i$ es la $i$-esima emdida de la potencia, $v_i$ es la $i$-esima  medida de lavelocidad del viento. La expresion anterior se puede reescribir de forma vectorial como
$$ \underset{\boldsymbol{\theta}}{ \text{min } } (\mathbf{P} - P(\mathbf{v},\boldsymbol{\theta}))^{\top}(\mathbf{P} - P(\mathbf{v},\boldsymbol{\theta})),
$$
donde $\mathbf{P}\in\mathbb{R}^n$ es el vector columna de todas las potencias medidas, $P(\mathbf{v},\boldsymbol{\theta})$ es un vector generado a partir de la evaluacion de la funcion $P(v,\boldsymbol{\theta})$ del vector de velocidades medidas, $\mathbf{v}$. Adicionalmente, cut-in speed es 2 m/s, cut-out speed es 18 m/s, y rated speed es 10 m/s. Rated power es 1800 kW.

In [35]:
def objfun(params):
  res = P-Power(params,v)
  return jnp.sum(res**2)

## Actividades

* A partir de la función objetivo y su gradiente, usar el método del gradiente más descendiente (Steepest Descent), para encontrar la solución al problema de optimización.
* Seleccionar un valor adecuado del paso ($\alpha$), de tal manera que el proceso de optimización converja a buenas soluciones.
* Comparar con el método de Newton.

In [47]:
key = random.key(3)
x0 = random.uniform(key, shape=(4,))
gradient = grad(objfun)
print(gradient(x0))

[-8.5316344e+04  9.4537753e-01 -7.1608758e-01 -1.1259868e+00]
